<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/nitky_Superswallow_7b_v0_1_instract_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install locale

ERROR: Could not find a version that satisfies the requirement locale (from versions: none)
ERROR: No matching distribution found for locale


In [2]:
!pip install -q transformers sentencepiece accelerate bitsandbytes

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(
    "nitky/Superswallow-7b-v0.1",
)
model = AutoModelForCausalLM.from_pretrained(
    "nitky/Superswallow-7b-v0.1",
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)

def generate(instruction_example, input_example):
  prompt = create_prompt(instruction_example, input_example)

  input_ids = tokenizer.encode(
      prompt,
      add_special_tokens=False,
      return_tensors="pt"
  )

  tokens = model.generate(
      input_ids.to(device=model.device),
      max_new_tokens=512,
      temperature=0.7,
      top_p=0.9,
      repetition_penalty=1.15,
      top_k=20,
      do_sample=True,
  )

  return tokenizer.decode(tokens[0], skip_special_tokens=True)

In [5]:
%%time
# Example usage
instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
input_example = "東京工業大学の主なキャンパスについて教えてください"
print(generate(instruction_example, input_example))

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
東京工業大学の主なキャンパスについて教えてください

### 応答:
東京工業大学は主に2つのキャンパスで構成されています。
1. 大田区の大岡山キャンパスとその周辺地区は、本学の中心的な拠点です。ここには学部、研究科、図書館などの主要施設が集まっており、約8,500名の学生が学んでいる。また、東工大発ベンチャーや起業家支援プログラムもこのエリアから生まれました。最寄り駅は東急目黒線「大岡山」駅。バスターミナルも近くにあり、渋谷、品川方面へのアクセスが便利です。
2. すずかけ台キャンパスは八王子市に位置し、理工学部の一部学科と先端科学技術研究センターが所在しています。約400人の学生が学ぶほか、共同利用機器室、実験棟、グラウンド、テニスコート等の設備が完備されています。最寄駅は京王電鉄相模原線「京王堀之内」駅から徒歩約3分。新宿まで約30分、横浜まで約60分の距離です。多摩都市モノレール「中央大学・明星大学」駅下車徒歩約7分。小田急電鉄「玉川学園前」駅下車タクシー約5分。町田市街地へのバス便も多く、交通利便性は良好です。
CPU times: user 18.6 s, sys: 361 ms, total: 18.9 s
Wall time: 23.7 s


In [6]:
%%time
# Example usage
instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
input_example = "アジアで有名な観光地と名物を詳しく述べてください。"
print(generate(instruction_example, input_example))

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
アジアで有名な観光地と名物を詳しく述べてください。

### 応答:
アジアは世界有数の人気観光地です。ここではいくつかの代表的な国や地域の特徴を紹介します。まず中国は世界最大の人口を抱える国であり、その歴史も非常に古いです。主な都市には北京・上海・香港などがあります。また日本は古くから先進技術と伝統文化を融合させた独自の発展を遂げています。東京は国際金融センターとして名高いです。インドは広大な国土面積を持つ多民族国家であり、さまざまな宗教や言語が共存しています。首都ニューデリーは巨大な都市圏を形成しており、経済面でも重要な拠点です。タイは東南アジアの中心部に位置し、美しいビーチリゾートが点在します。バンコクはかつての王都で今も多くの人々が住んでおり、寺院や市場等の見どころも豊富です。マレーシアはボルネオ島とマレー半島に跨る連邦立憲君主制国家です。クアラルンプールは同国の首都ですが、近年は近郊地区への開発も進みつつあります。インドネシアは南東アジア最西端の大規模な群島州です。ジャカルタは同国の首都かつ主要港湾都市であり、近代化した街並みが広がる一方、伝統的な文化も根付いています。フィリピンは東南アジア諸国連合加盟国であり、主にキリスト教信仰を有する国民が住むことから「神の家」とも呼ばれます。マニラは首都ながら、交通渋滞が深刻な問題となっています。シンガポールは東南アジアの小さな島国ですが、極めて豊かな経済を誇ることで知られます。オーストラリアは太平洋南部の大陸です。シドニーは同国の首都であり、イギリス植民地時代の遺産が数多く残されています。最後にニュージーランドはオセアニアの南西ポリネシアに位置する島国です。オークランドは同国の首都であり、マオリ族の遺跡や博物館が見所です。以上のようにそれぞれのエリアは多様性溢れる魅力が満載です。是非お気に入りの場所を見つけてみましょう。
CPU times: user 30.5 s, sys: 13.7 ms, total: 30.5 s
Wall time: 30.6 s


In [7]:
%%time
# Example usage
instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
input_example = "まどか☆マギカでは誰が一番かわいい?"
print(generate(instruction_example, input_example))

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
まどか☆マギカでは誰が一番かわいい?

### 応答:まどか☆マギカというアニメシリーズについてお話ししましょう。この作品に登場するキャラクターの中で最もかわいらしい人は、主人公の鹿目まどかです。彼女は優しく、思いやり深い性格を持ち、その外見も非常に可愛らしいものです。彼女は長い茶色髪を持つ美少女で、大きな青い瞳が特徴的です。また、彼女は魔法使いでもあり、さまざまな魔法を駆使して敵や困難を克服することができます。さらに、彼女はとても賢く、学校の成績も優秀です。これらの理由から、私は鹿目まどかが「まどか☆マギカ」の中で最も美しい人物だと思う。他のキャラクターも魅力的ですが、鹿目まどかは間違いなくトップ3に入るでしょう。
CPU times: user 11.6 s, sys: 122 ms, total: 11.7 s
Wall time: 11.8 s


In [8]:
%%time
# Example usage
instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
input_example = "日本で一番寒い地方はどこか教えてください。"
print(generate(instruction_example, input_example))

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
日本で一番寒い地方はどこか教えてください。

### 応答:
北海道は日本で最も寒い地域です。北海道は北極圏に位置し、冬は氷点下まで気温が下がることも珍しくありません。この地域に住む人々は厳しい寒さに適応するために厚手の衣類や暖房設備などを利用している傾向があります。また、北海道は国内でも雪が多い地域として知られています。特に札幌市は全国的に有名なスキーリゾートエリアとなっています。
CPU times: user 6.62 s, sys: 131 ms, total: 6.75 s
Wall time: 6.81 s


In [9]:
%%time
# Example usage
instruction_example = ""
# 参考wikipedia
input= (
    "以下の文章を20文字以下でまとめてください。\n\n"
    "赤血球は血液細胞の一つで色は赤く血液循環によって体中を回り、肺から得た酸素を取り込み、体のすみずみの細胞に運び供給する役割を担い、また二酸化炭素の排出にも関わる。赤血球の内部には鉄を含むタンパク質ヘモグロビンが充満しており、赤血球はヘモグロビンに酸素を取り込む。大きさは直径が7-8μm、厚さが2μm強ほどの両面中央が凹んだ円盤状であり、数は血液1μLあたり成人男性で450-650万個、成人女性で380-580万個程度で血液の容積のおよそ4-5割程度が赤血球の容積である。"
)
print(generate(instruction_example, input))

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:


### 入力:
以下の文章を20文字以下でまとめてください。

赤血球は血液細胞の一つで色は赤く血液循環によって体中を回り、肺から得た酸素を取り込み、体のすみずみの細胞に運び供給する役割を担い、また二酸化炭素の排出にも関わる。赤血球の内部には鉄を含むタンパク質ヘモグロビンが充満しており、赤血球はヘモグロビンに酸素を取り込む。大きさは直径が7-8μm、厚さが2μm強ほどの両面中央が凹んだ円盤状であり、数は血液1μLあたり成人男性で450-650万個、成人女性で380-580万個程度で血液の容積のおよそ4-5割程度が赤血球の容積である。

### 応答:赤血球は、体内をめぐる血液の中の重要な構成要素で、体全体へ酸素と二酸化炭素を運搬します。その核は赤い色素「ヘモグロビン」です。サイズは約7〜8マイクロメートルで、体の隅々まで酸素を届けることで健康維持に貢献します。
CPU times: user 5.77 s, sys: 339 ms, total: 6.11 s
Wall time: 6.1 s


In [10]:
%%time
# Example usage
instruction_example = ""
# 参考wikipedia
input = (
    "以下の文章は日本に関する文章です。国土の長さを答えてください。\n\n"
    "全長3500キロメートル以上にわたる国土は、主に日本列島および南西諸島・伊豆諸島・小笠原諸島などの弧状列島により構成される。大部分が温帯に属するが、北部や島嶼部では亜寒帯や熱帯の地域がある。地形は起伏に富み、火山地・丘陵を含む山地の面積は国土の約75%を占め、人口は沿岸の平野部に集中している。国内には行政区分として47の都道府県があり、日本人（大和民族・琉球民族・アイヌ民族・外国系諸民族）と外国人と数百人程度の無国籍者が居住し、日本語を通用する"
)
print(generate(instruction_example, input))

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:


### 入力:
以下の文章は日本に関する文章です。国土の長さを答えてください。

全長3500キロメートル以上にわたる国土は、主に日本列島および南西諸島・伊豆諸島・小笠原諸島などの弧状列島により構成される。大部分が温帯に属するが、北部や島嶼部では亜寒帯や熱帯の地域がある。地形は起伏に富み、火山地・丘陵を含む山地の面積は国土の約75%を占め、人口は沿岸の平野部に集中している。国内には行政区分として47の都道府県があり、日本人（大和民族・琉球民族・アイヌ民族・外国系諸民族）と外国人と数百人程度の無国籍者が居住し、日本語を通用する

### 応答:「およそ」3,802km
CPU times: user 1.93 s, sys: 333 ms, total: 2.27 s
Wall time: 2.26 s


In [11]:
%%time
# Example usage
instruction_example = ""
# 参考wikipedia
input = (
    "以下の文章は日本に関する文章です。国土の長さ（全長）を答えてください。\n\n"
    "全長3500キロメートル以上にわたる国土は、主に日本列島および南西諸島・伊豆諸島・小笠原諸島などの弧状列島により構成される。大部分が温帯に属するが、北部や島嶼部では亜寒帯や熱帯の地域がある。地形は起伏に富み、火山地・丘陵を含む山地の面積は国土の約75%を占め、人口は沿岸の平野部に集中している。国内には行政区分として47の都道府県があり、日本人（大和民族・琉球民族・アイヌ民族・外国系諸民族）と外国人と数百人程度の無国籍者が居住し、日本語を通用する"
)
print(generate(instruction_example, input))

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:


### 入力:
以下の文章は日本に関する文章です。国土の長さ（全長）を答えてください。

全長3500キロメートル以上にわたる国土は、主に日本列島および南西諸島・伊豆諸島・小笠原諸島などの弧状列島により構成される。大部分が温帯に属するが、北部や島嶼部では亜寒帯や熱帯の地域がある。地形は起伏に富み、火山地・丘陵を含む山地の面積は国土の約75%を占め、人口は沿岸の平野部に集中している。国内には行政区分として47の都道府県があり、日本人（大和民族・琉球民族・アイヌ民族・外国系諸民族）と外国人と数百人程度の無国籍者が居住し、日本語を通用する

### 応答:約38,691km
CPU times: user 1.84 s, sys: 308 ms, total: 2.15 s
Wall time: 2.21 s


In [12]:
%%time
# Example usage
instruction_example = ""
# 参考wikipedia
input = (
    "以下の文章はどこの国について述べていますか。\n\n"
    "全長3500キロメートル以上にわたる国土は、主に日本列島および南西諸島・伊豆諸島・小笠原諸島などの弧状列島により構成される。大部分が温帯に属するが、北部や島嶼部では亜寒帯や熱帯の地域がある。地形は起伏に富み、火山地・丘陵を含む山地の面積は国土の約75%を占め、人口は沿岸の平野部に集中している。国内には行政区分として47の都道府県があり、日本人（大和民族・琉球民族・アイヌ民族・外国系諸民族）と外国人と数百人程度の無国籍者が居住し、日本語を通用する"
)
print(generate(instruction_example, input))

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:


### 入力:
以下の文章はどこの国について述べていますか。

全長3500キロメートル以上にわたる国土は、主に日本列島および南西諸島・伊豆諸島・小笠原諸島などの弧状列島により構成される。大部分が温帯に属するが、北部や島嶼部では亜寒帯や熱帯の地域がある。地形は起伏に富み、火山地・丘陵を含む山地の面積は国土の約75%を占め、人口は沿岸の平野部に集中している。国内には行政区分として47の都道府県があり、日本人（大和民族・琉球民族・アイヌ民族・外国系諸民族）と外国人と数百人程度の無国籍者が居住し、日本語を通用する

### 応答:日本
CPU times: user 1.21 s, sys: 241 ms, total: 1.45 s
Wall time: 1.49 s
